# Unsupervised Learning: 
# comparazione tra algoritmi di clustering (k-mean vs GMM)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

Parlando di algoritmi di Machine Learning, questi vengono principalmente suddivisi in due grandi famiglie: apprendimento supervisionato e apprendimento non supervisionato. 
Il primo passo per la soluzione di un problema é capire se il modello che andremo ad implementare utilizzerá degli algoritmi di apprendimento automatico facenti parte del primo gruppo o del secondo (anche non di rado potrebbero volerci entrambi). Per farlo, occorrerá studiare il set di dati che ci viene fornito, che di solito si compone di una serie di caratteristiche (features) e, opzionalmente, una lista di output. 

Per rendere le cose semplici, possiamo dire che se nel nostro dataset sono presenti delle variabili di output dovremo orientarci su uno o piú algoritmi di apprendimento supervisionato, altrimenti la strada da percorrere sará quella dell'apprendimento non supervisionato. 
Facciamo un paio di esempi: 
1) Dallo storico dei dati di vendita di un negozio vorrei predire le vendite nel prossimo mese. Questo é un esempio di apprendimento supervisionato;
2) Dato un grande database di clienti (con il loro relativo storico di acquisti) vorrei definire dei gruppi in base a comportamenti di acquisto simili. Non conoscendo a priori queste proprietá comportamentali, utilizzeró degli algoritmi di apprendimento non supervisionato.

Tra gli algoritmi di apprendimento non supervisionato, quelli piú comuni sono gli algoritmi di clustering, usati appunto per raggruppare i dati di storico in varie classi. Lo scopo é individuare i centroidi di questi cluster, in modo da poter calcolare con semplicitá a quale classe appartiene il prossimo dato che viene buttato in pasto al nostro modello. I due algoritmi principali usati per questo scopo sono: K-Means e i Gaussian Mixture Models (GMM).

## Fase 1: Dataset
### Genero un dataset di blobs randomici

Lo script che segue genera un grafico con dei punti (800) sparsi in modo casuale. 
Per le nostre elaborazioni, ipotizziamo di dividere il nostro dataset in tre cluster distinti.
La funzione make_blobs prende come input il numero di oggetti che vogliamo generare e il numero di cluster su cui dividerli (n_samples e centers). Come output, ci fornirá X, un array bidimensionale (ogni elemento rappresenta le coordinate x e y dell' oggetto generato casualmente in modo da poterlo disegnare come punto su un piano cartesiano), e un array monodimensionale y_true con il numero del cluster al quale appartiene quell' oggetto.

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=800, centers=4)
plt.scatter(X[:, 0], X[:, 1]);

## K-means

Il primo algoritmo di clustering non supervisionato che andremo ad analizzare si chiama K-Means. Il suo compito é quello di assegnare a ogni oggetto di un set di dati multidimensionale (bidimensionale in questo caso) il suo cluster di appartenenza (sapendo a priori il numero di cluster su cui dividere i dati). Per farlo, utilizza i seguenti concetti:
1) il centroide é la media aritmetica tra tutti gli oggetti facenti parte di quel cluster
2) un oggetto facente parte di un cluster é piú vicino al centroide di quel cluster rispetto a tutti gli altri centroidi.

Usando la tipica API di predizione di scikit-learn, lasceremo all'algoritmo il compito di trovare il centro di ogni cluster e di conseguenza assegnare ogni punto al cluster con il centroide piú vicino. 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.5);

Diamo un'occhio alla performace dell' algoritmo:

In [ ]:
%timeit -n 1000 kmeans.predict(X)

198 microsecondi. Non é un errore. Se vi state chiedendo come abbia fatto a fare tanti calcoli in cosí poco tempo la risposta é che in realtá non li ha fatti, almeno non tutti. Questo é l'algoritmo su cui si basa k-mean:

- Setta i centri dei cluster a caso
- Ripete fino alla convergenza:
    - assegna i punti al cluster con il centroide piú vicino
    - ricalcola i centri dei cluster usando la media dei punti di ogni cluster 

Spesso, a ogni iterazione dell'algoritmo, il risultato sará una sempre migliore stima delle caratteristiche del cluster.

## Gaussian Mixture Models

In [ ]:
from sklearn.mixture import GMM
gmm = GMM(n_components=4).fit(X)
labels = gmm.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

Come per K-means, ho usato l'API di scikit-learn usando l'algoritmo chiamato GMM (Gaussian Mixture Models). 

Il risultato é simile, anche se non identico, infatti:

In [ ]:
print("Gli oggetti sono assegnati agli stessi cluster nei due algoritmi? {}".format(all(labels == y_kmeans)))
print("Quanti sono quelli diversi (su 800)? {}".format(len([x for x, y in zip(labels, y_kmeans) if x != y])))

Ma quanto ci mette a fare i calcoli?

In [ ]:
%timeit -n 1000 gmm.predict(X)

Come possiamo osservare, questo algoritmo é molto piú lento di k-means. A sua discolpa, GMM fa qualche considerazione in piú, e ci aiuta a calcolare i cluster in alcuni casi in cui k-means non ci riesce. Quali sono questi casi? 

Guardiamo il dataset con l'occhio umano: mentre per alcuni punti possiamo dire con esattezza a quale cluster appartiene, per altri il confine diventa piú sfocato. K-means non conosce questo significato, quindi cercheró di spiegare in modo semplice come ragiona: immaginate di disegnare un cerchio avente centro il centroide del cluster e come raggio la distanza tra il centroide e l'oggetto piú lontano di quel cluster. Per k-mean, ogni oggetto al di fuori di questo cerchio, non fa parte del cluster.

Facciamo un esempio grafico:

In [ ]:
from scipy.spatial.distance import cdist

def plot_kmeans(kmeans, X, n_clusters=4, rseed=0, ax=None):
    labels = kmeans.fit_predict(X)

    # plot the input data
    ax = ax or plt.gca()
    ax.axis('equal')
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)

    # plot the representation of the KMeans model
    centers = kmeans.cluster_centers_
    radii = [cdist(X[labels == i], [center]).max()
             for i, center in enumerate(centers)]
    for c, r in zip(centers, radii):
        ax.add_patch(plt.Circle(c, r, fc='#CCCCCC', lw=3, alpha=0.5, zorder=1))

In [ ]:
plot_kmeans(kmeans, X)

Questo approccio non funziona su un set di dati fatto per esempio cosí:

In [ ]:
rng = np.random.RandomState(13)
X_stretched = np.dot(X, rng.randn(2, 2))

kmeans = KMeans(n_clusters=4, random_state=1)
plot_kmeans(kmeans, X_stretched)

In questo caso, piú che un raggruppamento su una figura circolare, ci saremmo aspettatti un raggruppamento su base ellittica per esempio. Guardiamo la differenza con l'elaborazione di GMM:

In [ ]:
from matplotlib.patches import Ellipse

gmm = GMM(n_components=4, covariance_type='full', random_state=42)

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 5):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))

def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covars_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
        
plot_gmm(gmm, X_stretched)